I use this notebook to generate results on the  official testset using my best models

#Setup

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'test-281700'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

gs://spotify_asr_dataset/


In [ ]:
#download model
bucket_name = 'spotify_asr_dataset'
!gsutil -m cp -r gs://{bucket_name}/t5-model-66242.zip /content/

Copying gs://spotify_asr_dataset/t5-model-66242.zip...
/ [1/1 files][789.1 MiB/789.1 MiB] 100% Done  38.2 MiB/s ETA 00:00:00           
Operation completed over 1 objects/789.1 MiB.                                    


In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 890kB 37.3MB/s 
     |████████████████████████████████| 3.0MB 35.9MB/s 
     |████████████████████████████████| 1.1MB 40.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b00284178be3a8a9c7f566b1e1819f18fe30af8da7f1cba8600bfb825991d4de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!unzip t5-model-66242.zip

Archive:  t5-model-66242.zip
   creating: t5-model-66242/
  inflating: t5-model-66242/spiece.model  
  inflating: t5-model-66242/special_tokens_map.json  
  inflating: t5-model-66242/config.json  
  inflating: t5-model-66242/tokenizer_config.json  
  inflating: t5-model-66242/pytorch_model.bin  


In [ ]:
import pandas as pd
testset = pd.read_csv('testset.csv')
metadata = pd.read_csv('metadata-summarization-testset.tsv', sep='\t')

In [ ]:
testset.head(2)

,episode_id,transcript
0,spotify:episode:74t5WREXUbhEKNI89CNSkL,Welcome back to another episode of tuxedo tim...
1,spotify:episode:5fG4VlWnWwzAt6mSs0H7lY,"What's up, guys? This episode of the podcast ..."


In [ ]:
len(testset)

1027

In [ ]:
metadata.head(2)

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix
0,spotify:show:015DbLwcXu2fK7e9jIfbFo,The Tuxedo Time Podcast,"Welcome to a special edition of Tuxedo Time, P...",Becki and Chris,['en'],https://anchor.fm/s/e342db8/podcast/rss,spotify:episode:74t5WREXUbhEKNI89CNSkL,Ep.05 Why We Don't Sell Presets,Today on the podcast we go on a JOURNEY! We ta...,54.27120,show_015DbLwcXu2fK7e9jIfbFo,74t5WREXUbhEKNI89CNSkL
1,spotify:show:01DbRiALDPdvZdoiY8yQL6,The Wavelengths Podcast,Ever wanted a podcast from your three favorite...,The Wavelengths Podcast,['en'],https://anchor.fm/s/14c6aa84/podcast/rss,spotify:episode:5fG4VlWnWwzAt6mSs0H7lY,How to Grow as an Esports Player - The Wavelen...,Ever wanted a podcast from your three favorite...,77.50085,show_01DbRiALDPdvZdoiY8yQL6,5fG4VlWnWwzAt6mSs0H7lY


In [ ]:
test = testset.merge(metadata, left_on='episode_id', right_on='episode_uri')
test = test[['episode_id', 'episode_filename_prefix', 'show_filename_prefix', 'episode_name', 'episode_description', 'transcript']] #useful later
test = test[450:600] #takes too long, run time disconnects etc
test.head(2)

,episode_id,episode_filename_prefix,show_filename_prefix,episode_name,episode_description,transcript
450,spotify:episode:62O9hxuV4xnaZ9xbiPW5Ri,62O9hxuV4xnaZ9xbiPW5Ri,show_3HWnUfK8VXBXVnrtJS7eAR,EP 0098 - STOP Checking In With Yourself!,Full show notes for this episode are at https:...,Hello there before we get started on today's ...
451,spotify:episode:70XJryeCkTbxfSDXhiwvkk,70XJryeCkTbxfSDXhiwvkk,show_3i7Oegu4582Fg03tr4UEOd,From soccer to entrepreneur - W/ Mikey Moon,Mikey Moon (@Mikeymoonfitness) went from playi...,"Hi guys, welcome to today's podcast episode o..."


In [ ]:
len(test)

150

#Inference

In [ ]:
"""
clean_up Post processes generated summaries so that all promotional material are removed
"""

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import string

def clean_up(text):
  head, _ , _ = text.partition(' ---')
  head = head.strip()
  first_letter = head[0].capitalize()
  head = first_letter + head[1:]
  if head[-1] in string.punctuation:
    pass
  else:
    head += '.'
  return head

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# # # Setting up the device for GPU usage
# from torch import cuda
# device = 'cuda' if cuda.is_available() else 'cpu'

tokenizer = T5Tokenizer.from_pretrained('/content/t5-model-66242')
model = T5ForConditionalGeneration.from_pretrained('/content/t5-model-66242')


def t5_inference(transcript):
  threshold = 6500
  t5_form = 'summarize: ' + transcript
  tokenized_text = tokenizer.encode(t5_form, return_tensors="pt")
  if len(tokenized_text[0]) > threshold:
    #run out of RAM/crashes on large number of tokens
    revised_text = sent_tokenize(t5_form)
    length = len(revised_text)
    final_text = revised_text[:int(length/2)] #divide transcript in half to manage the number ot tokens generated
    text = ' '.join(final_text)
    return t5_inference(text)
  summary_ids = model.generate(tokenized_text, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  output = clean_up(output)
  return output

In [ ]:
test['summary'] = test.apply(lambda row: t5_inference(row['transcript']), axis=1)
test.to_csv('test_results_600.csv', index=False)
!gsutil -m cp -r /content/test_results_600.csv gs://{bucket_name}/

Token indices sequence length is longer than the specified maximum sequence length for this model (7015 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3287 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5622 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (10276 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5012 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length f

Copying file:///content/test_results_600.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/4.4 MiB.                                      
